Setup

In [1]:
from pathlib import Path

from ataraxis_base_utilities import ensure_directory_exists

from sl_forgery.utils.dataclass import ProjectData, TargetGroup
from sl_forgery.analysis.plotting import Plotting, TraceType
from sl_forgery.analysis.processing import Processing
from sl_forgery.analysis.create_manifest import make_manifest_from_structure

project_root = Path.cwd().resolve().parents[2]

# Working directory is a folder called slf_data at the same level as the project. Within my local slf_data folder I have downloaded the TM_06_pilot data
work_dir = (project_root / ".." / "slf_data").resolve()

ensure_directory_exists(work_dir)

project_name = "TM_06_pilot"
assert (work_dir / project_name).is_dir()

# TM_06_pilot does not have a manifest file so I had to generate one. In general, this code is not necessary as all projects will have a project manifest
manifest_path = work_dir / f"{project_name}_manifest.feather"
if not manifest_path.is_file():
    rows = [
        {
            "animal": 6,
            "session": "2025-06-23-13-32-06-980761",
            "type": "MESOSCOPE_EXPERIMENT",
            "complete": True,
            "integrity": True,
            "dataset": True,
        },
        {
            "animal": 6,
            "session": "2025-06-24-13-17-47-781337",
            "type": "MESOSCOPE_EXPERIMENT",
            "complete": True,
            "integrity": True,
            "dataset": True,
        },
        {
            "animal": 6,
            "session": "2025-06-25-16-35-01-581331",
            "type": "MESOSCOPE_EXPERIMENT",
            "complete": True,
            "integrity": True,
            "dataset": True,
        },
        {
            "animal": 6,
            "session": "2025-06-26-12-57-38-495382",
            "type": "MESOSCOPE_EXPERIMENT",
            "complete": True,
            "integrity": True,
            "dataset": True,
        },
        {
            "animal": 6,
            "session": "2025-06-27-12-44-58-770644",
            "type": "MESOSCOPE_EXPERIMENT",
            "complete": True,
            "integrity": True,
            "dataset": True,
        },
    ]
    manifest_path = make_manifest_from_structure(project_name, rows, work_dir)

# See src\sl_forgery\analysis\example_filter.yaml for full example of how the filter file should be structured
filter_path = project_root / r"src\sl_forgery\analysis\tm6_filter.yaml"

c:\Users\jacob\miniforge3\envs\slf_dev_win\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Create Dataclass

In [2]:
data = ProjectData.create(
    project_name=project_name, manifest_path=manifest_path, filter_path=filter_path, working_directory=work_dir
)

Plot Sessions

In [3]:
fig = Plotting.plot_session("single_day", 4, data.get_session_data("2025-06-23-13-32-06-980761"))

In [4]:
fig = Plotting.plot_multi_session(
    target_group="single_day", trace_type=TraceType.PER_TRIAL, cell=4, animal=data.get_mouse(6)
)

Umaps

In [5]:
fig = Plotting.plot_umap("single_day", data.get_session_data("2025-06-23-13-32-06-980761"))

In [6]:
#  Compute the UMAPS and save them to the dataclass
for session_name in data.manifest.get_sessions(animal=6):
    Processing.compute_single_session_umap("single_day", data.get_session_data(session_name))

In [7]:
fig = Plotting.plot_all_single_session_umaps(
    target_group=TargetGroup.SINGLE_DAY, animal=data.get_mouse(6), save_path=work_dir / "sample_plot.html"
)